In [1]:
import numpy as np
from tqdm import tqdm
import multiprocessing
import json

from skimage import io
from skimage.filters import meijering, apply_hysteresis_threshold, gaussian, median
from skimage.morphology import binary_closing, disk, remove_small_objects, skeletonize
from skan import Skeleton, summarize

import warnings
warnings.filterwarnings('ignore')

In [2]:

file_path = '../exp-data/20251004/40V_10X/'
output_file = 'test'

def framework(image):
    # Turning image to float format
    if image.dtype!= 'float':
        image = image.astype(float) / 255.0

    # Step-1 : Applying noise filters:
    denoised_image = median(image, footprint=np.ones((3, 3)))

    # Step-2 : Detecting defect strings as ridge-like structure
    ridge_map = meijering(denoised_image, sigmas = range(1, 4, 1), black_ridges = True)
    
    # Step-3 : Segmenting with hysteresis
    binary_ridge_map = apply_hysteresis_threshold( ridge_map, low = 0.08, high = 0.2)

    # Step-4 : Refining and cleaning 
    closed_map = binary_closing(binary_ridge_map, footprint = disk((3)))
    cleaned_map = remove_small_objects(closed_map, min_size = 16)

    # Step-5 : Skeletonizing and measuring total length
    skleton = skeletonize(cleaned_map)
    skeleton_obj = Skeleton(skleton)
    summary_df  = summarize(skeleton_obj)
    total_length = summary_df['branch-distance'].sum()
    
    return total_length
    

def perform_processing(idx):
    t_frame = idx/100
    fig_name = file_path + "/{:.2f} s.tif".format(t_frame)
    try: 
        img = io.imread(fig_name, as_gray=True)
        out = framework(img)
        return [t_frame, out]
    except:
        return


## 4. processing::==============================================================================================
if __name__ == '__main__':
    cores = multiprocessing.cpu_count()   # Define number of cores for multiprocessing.
    
    with multiprocessing.Pool(cores) as pool:
        outs = [result for result in tqdm(pool.imap(perform_processing, range(200, 221, 1)), total=20) 
                if result is not None]


class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

output_file = '../results/' + output_file

with open('%s.json'%output_file, 'w') as file:
    json.dump(outs, file, indent=4, cls=NpEncoder)


21it [00:42,  2.05s/it]                        


In [3]:
with open('../results/test.json') as f:
    data = json.load(f)

In [4]:
data

[[2.0, 15369.697320904515],
 [2.01, 15309.229283472978],
 [2.02, 15328.773439350267],
 [2.03, 14964.90843229906],
 [2.04, 15058.636354360418],
 [2.05, 14470.746960691575],
 [2.06, 15145.60186647501],
 [2.07, 14849.724666114984],
 [2.08, 14669.001794687509],
 [2.09, 14935.16831692888],
 [2.1, 14529.830221812255],
 [2.11, 14433.545950564792],
 [2.12, 14487.344940438019],
 [2.13, 14488.332747129196],
 [2.14, 14822.996744053627],
 [2.15, 14542.504320004453],
 [2.16, 13970.058577149503],
 [2.17, 13910.673800838653],
 [2.18, 13500.382386916255],
 [2.19, 14178.639312953246],
 [2.2, 14089.129644961367]]